In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 47.3 MB 1.6 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.5 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595725 sha256=37877421052e35849934373447f0f9288e5233e9f4657a971eb167213ed52bc5
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.4 MB/s 
     |████████████████████████████████| 769 kB 5.3 MB/s 
     |████████████████████████████████| 895 kB 44.1 MB/s 
     |████████████████████████████████| 3.0 MB 35.3 MB/s 


In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-l_tb83np
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-l_tb83np
     |████████████████████████████████| 4.9 MB 5.4 MB/s 
     |████████████████████████████████| 3.4 MB 36.7 MB/s 
     |████████████████████████████████| 3.3 MB 39.2 MB/s 
     |████████████████████████████████| 61 kB 496 kB/s 
     |████████████████████████████████| 596 kB 50.3 MB/s 
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=13136 sha256=c3e916988615be84583d6ebaf6eb2ba7a1397df2257142035c8fd968e8db97b6
  Stored in directory: /tmp/pip-ephem-wheel-cache-ueih9_zq/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: tokenizers
    Found existi

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

.cache/kobert_v1.zip[██████████████████████████████████████████████████]
.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
chatbot_data = pd.read_excel('/content/drive/MyDrive/AI/AI_study/data set_save/한국어 감정 정보가 포함된 단발성 대화 데이터셋/한국어_단발성_대화_데이터셋.xlsx')

In [ ]:
len(chatbot_data)

38594

In [ ]:
chatbot_data.sample(n=10)

,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468
16771,민병헌이 해커한테 공던진거 아직도 기억난다.. 그게 선수가 할 행동인지.. 쯧쯧,분노,NaN,NaN,NaN,NaN,NaN
24043,솔찍한건지 무시하는건지 너무 솔찍한 정치인인가?,중립,NaN,NaN,NaN,NaN,NaN
24609,설마가 사람 죽인다,중립,NaN,NaN,NaN,NaN,NaN
1364,외로워서...너무할일이없어서...취정게에 뻘낚시글을 썼다가 글삭에 경고크리먹었음.,공포,NaN,NaN,NaN,NaN,NaN
7327,너넨 어떻게 마음 정리했어?,놀람,NaN,NaN,NaN,NaN,NaN
36387,저년 혓바닥을 짤라야 바른말 할려나,혐오,NaN,NaN,NaN,NaN,NaN
19949,삼성 재고만 해도 100만대가 있었다...,슬픔,NaN,NaN,NaN,NaN,NaN
1723,근데 이남자가 안경을 쓰고 그래서 인상도 그닥 좋지 않고 좀 무서운데..,공포,NaN,NaN,NaN,NaN,NaN
3040,나라가 망할거 같아 두렵다,공포,NaN,NaN,NaN,NaN,NaN
11295,역시 문학작품은 그냥만들어지는게 아닌거야,놀람,NaN,NaN,NaN,NaN,NaN


In [ ]:
chatbot_data.loc[(chatbot_data['Emotion'] == "공포"), 'Emotion'] = 0  #공포 = 0
chatbot_data.loc[(chatbot_data['Emotion'] == "놀람"), 'Emotion'] = 1  #놀람 = 1
chatbot_data.loc[(chatbot_data['Emotion'] == "분노"), 'Emotion'] = 2  #분노 = 2
chatbot_data.loc[(chatbot_data['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 = 3
chatbot_data.loc[(chatbot_data['Emotion'] == "중립"), 'Emotion'] = 4  #중립 = 4
chatbot_data.loc[(chatbot_data['Emotion'] == "행복"), 'Emotion'] = 5  #행복 = 5
chatbot_data.loc[(chatbot_data['Emotion'] == "혐오"), 'Emotion'] = 6  #혐오 = 6

In [ ]:
data_list = []
for q, label in zip(chatbot_data['Sentence'], chatbot_data['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
print(len(data_list))
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[24000])
print(data_list[30000])
print(data_list[-1])

38594
['언니 동생으로 부르는게 맞는 일인가요..??', '0']
['기술적으로도 아직도 해체해서 다시 완벽히 돌려놓는게 어려운데 해체를한다고?', '1']
['당연히 그렇게 해야지 우리나라도 판매를 중단하라', '2']
['그거들은 뒤부터 미치겠어요...', '3']
['최악의 상황중 그나마 나은 방법이네. 기분은 잡치겠지만', '4']
['  요리하는것이 숙제하는것처럼 힘든저에게 용기나게 해주시고 할수 있을것같은 희망을 주셔서감사합니다!!', '5']
['와이프도 그렇고 댓글 다 볼텐데 이휘재 좀 하차 하라고 전해주세요', '6']


In [ ]:
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [ ]:
print(len(dataset_train))
print(len(dataset_test))
print(dataset_train[0])
print(dataset_test[0])

28945
9649
['그냥 멋져유 ~요리할땐 더 멋져유~기부했다고 들었을땐 더더더 멋져부러유~~^^', '5']
[" '어이없는 실책' 이라고 아나운서가 예기하더군요.", '0']


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
data_train[0]

(array([   2, 1189,  517, 6188, 7245, 7063,  517,  463, 3486, 7836, 5966,
        1698,  517, 6188, 7245, 7063,  517,  463, 1281, 7870, 1801, 6885,
        7088, 5966, 1698, 5837, 5837,  517, 6188, 7245, 6398, 6037, 7063,
         517,  463,  517,  463,  517,  364,  517,  364,    3,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(42, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 5)

In [ ]:
data_test[1]

(array([   2, 1665, 6993,  862, 5561, 6553, 1198, 5439,  736, 5377, 6896,
        6999,  633,    3,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(14, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 0)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
!conda install pytorch torchvision cudatoolkit=10.1 -c pytorch

/bin/bash: conda: command not found


In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
train_dataloader

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/453 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.9782887697219849 train acc 0.109375
epoch 1 batch id 201 loss 1.539293885231018 train acc 0.2218594527363184
epoch 1 batch id 401 loss 1.2744337320327759 train acc 0.3298784289276808
epoch 1 train acc 0.347049084534476


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 9 batch id 1 loss 0.052405938506126404 train acc 0.984375
epoch 9 batch id 201 loss 0.1112338975071907 train acc 0.9420864427860697
epoch 9 batch id 401 loss 0.2321244329214096 train acc 0.9454099127182045
epoch 9 train acc 0.9471578366445916


  0%|          | 0/151 [00:00<?, ?it/s]

epoch 9 test acc 0.5343500810920394


  0%|          | 0/453 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.04281396418809891 train acc 1.0


In [ ]:
import os
os.chdir('/content/drive/MyDrive/AI/할수있다 with AI/')
os.getcwd()

In [ ]:
path = '/content/drive/MyDrive/AI/할수있다 with AI/'
torch.save(model, path + 'kobert_jinro.pt')

In [ ]:
torch.save(model.state_dict(), 'kobert_jinro_state_dict.pt')

In [ ]:
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, 'kobert_jinro_all.tar')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('/content/drive/MyDrive/AI/할수있다 with AI/')

model1 = torch.load('kobert_jinro.pt')  
model1.load_state_dict(torch.load('kobert_jinro_state_dict.pt'))  

checkpoint = torch.load('kobert_jinro_all.tar')
model1.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer'])

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [ ]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model1.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model1(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                return 'bad'
            elif np.argmax(logits) == 1:
                test_eval.append("놀람이")
                return 'good'
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
                return 'bad'
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
                return 'bad'
            elif np.argmax(logits) == 4:
                test_eval.append("중립이")
                return 'good'
            elif np.argmax(logits) == 5:
                test_eval.append("행복이")
                return 'good'
            elif np.argmax(logits) == 6:
                test_eval.append("혐오가")
                return 'bad'
        
        print(test_eval[0] + " 느껴집니다.")

In [ ]:
end = 1
bad = 0
good = 0

while end == 1 :
    sentence = input("말을 입력해주세요 : ")
    if sentence == "0" :
        break
    if predict(sentence) == 'bad':
        bad += 1
    else:
        good += 1
        
    predict(sentence)
    print("\n")
    
print(bad)
print(good)

말을 입력해주세요 : 안녕


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))




말을 입력해주세요 : 안녕


말을 입력해주세요 : 안녕


말을 입력해주세요 : 안녕


말을 입력해주세요 : 0
0
4


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/AI/AI_study/data set_save/out.csv')

In [ ]:
for row_index, row in data.iterrows():
    print(row_index)
    print(row['comment'])

0
혼인빙자… 말도 안되는 단어 아니냐 사실왜 ? 앞으론 사귈 때 혼인 할건지 아닌지 각서라도 쓰고 만나지 그러냐?
1
불찰은 개뿔~ 할때는 좋았겠지
2
남들다 하는 연애 하고 이별. 전남친 잘나가는 꼴 못보는 이혼녀가 부리는 개수작에 놀아나는 한심한 국민성
3
여자든 남자든 사람 잘 만나야 한다는 걸 또 한번 느낀다
4
됐고. 쨔져. 딱 이게 니 수준으로 말해주는거다. 착한척하다 더뜨면 너만힘들어. 개버릇 남주니? 하던대로 살아
5
뿌란씨대로 걷는게 새상ㅇㅣ치
6
폭로자분도 이상하신듯 사과를 받았고 오해를 풀었다고하지만 폭로의글로인해 그파급력은 지금 어마어마하다 사과받고 끝낼일이었다면 그냥따로 만나지 좀 그렇다
7
청춘기록에서 처럼 자기 배우가 뜨면 이용하려고 하고 인기 없으면 밟아버리려하는 곳이 아직도 존재한다면 그런 인간은 생매장되야죠! 그런 회사는 망해야 맞습니다. 이건 바로 고소김이에요!! 개인정보 위반법. 그리고 사생활 침범. 강력대응 할 수 있어요. 당신도 잘못이 있으니 사실을 바른 눈으로 볼 수 없는거죠! 빨리 갯마을 차차차랑 상의해서 고소하시고 피해보상을 폭로자에게 청구하세요. 그리고 자신의 잘못은 책임져야죠. 그게 사람이 할 도리입니다.
8
선호씨 그냥 그건 둘 다 똑같은 것들이에요. 뭐가 미안합니까? 낙태를 조용하면 낙태가 저절로 되는 일인가요? 아기를 갖고 싶었다면 그냥 낳았겠죠!!! 지금 사과 할 곳은 그 정신나간 여자가 아니라 갯미을차차차의 작가 감독모든 스탭과 배우감독이죠! 당신들의 추잡했던 과거사로 피해를 본건 갯마을 차차차 아닌가요? 그리고 그런여자는 헤어질만한 여자죠! 만약 낳고 스스로 독립적으로 키우려했던 사름이얐다면 버린 놈이 등신이였겠지만! 정신차리세요! 그리고 이렇게 타이밍 좋게 딱딱 맞춰서 보도자료를 뿌릴 수 있는 힘의 근원을 찾으세요.!
9
저정도와꾸면 뭐 그래도 선호야 토리는다했어야지지금이라도 사과하라 며친한테
10
요즘도 혼인빙자라는 말이 있나?결혼 해줄? 줄 알고 같이 잤다는건가?여자도 자기 행동에 책임을 져야지

In [ ]:
for row_index, row in data.iterrows():
    #print(row_index)
    #print(row['comment'])

    sentence = row['comment']
    predict(sentence)
    print('\n')
    
    if predict(sentence) == 'bad':
        bad += 1
    else:
        good += 1
    
print(bad)
print(good)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 232, in _feed
    close()
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor


Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 232, in _feed
    close()
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/AI/AI_study/data set_save/jtbc.csv')

for row_index, row in data.iterrows():
    print(row_index)
    print(row['comment'])

for row_index, row in data.iterrows():
    #print(row_index)
    #print(row['comment'])

    sentence = row['comment']
    predict(sentence)
    print('\n')
    
    if predict(sentence) == 'bad':
        bad += 1
    else:
        good += 1
    
bad1 = bad
good1 = good

0
성인남녀 만나서 그랬을수도있다고본다.. 하지만 그후엔 그여자분을 보듬어줬어야했고 이해시켰어야했고 한을 남기지말았어야지.. 요즘이 어떤세상인데.. 앞으로 살아갈날이 더많아서 어떻게 마주칠지도 모르고 또 세상좁다.. 부끄럽지않게 남한테 상처주며 살지말아야지
1
여자입장에서는 아이도 지우고 남자에겐 이별통보받고 고통스럽겠지만..연애하다보면 한명은 너무 좋아서 발목이라도 잡고싶은데 한명은 그냥 연애만하고 결혼할 배우자라 생각안한 경우도 많다.. 그냥 그런 케이스..낙태는 마음아프겠지만 한명이라도 아이를 기를수없다고 판단했다면 낳아서 이혼하고 상처주는거보다 나은 선택일수 있다 혼전임신해서 책임진다며 덜컥 결혼해 아이낳고 이혼하고 그런 가정 많다 그게 아이한테든 본인들한테든 더 안좋다고봄
2
차라리 깔끔하게 사과하니까 더 낫네요.
3
좋게 만나고 좋게 헤어지자
4
아줌마들 김선호 쉴드 그만치세요 개역겨워ㅠㅠㅠㅠ
5
여자가 낙태한것 때문에 이게 문제가 되는게 아니지..나도 남자고 페미 싫어하는데..그 과정이 역겨웠던게 문제지. 온갖 거짓말이랑 사탕발림으로 상대가 원하지도 않는 낙태를 하기 하고 하자마자 버린게 문제지. 내동생이 저렇게 당했으면 찢어 죽인다.
6
같은 남자가 봐도 하..정말 쓰레기짓을 아무렇지 않게 하고 방송에서 이미지 메이킹을 했다는게 소름끼친다….그렇게 자기 성공이 중요했으면 만나는 동안 최소한 조심했어야지..본인이 싸질러 놓고 그 책임을 상대가 오롯이 지게 한것 자체가 쓰레기다..일말의 미안함도 안보이고..그냥 어떻게던 위기만 피하려고 하니..
7
팬들 보아라.대체 상대방여자가 어떤사람인게 뭐가 중요한가?긴 무명 견디고 깨끗한 이미지로 이미지메이킹 성공하여 이제 라이징스타로 올라왔는데 한창 1박으로 인기 급상승 중이던 2020년에 만나 몇달만에 임신에 중절까지 하게 했다는 것. 한창 자기 몸값 올라가는 시기에 사귀기 시작했다는 여친인데 생각 없이 피임도 안하고 아이 생기니까 자기 앞날 구구절절 핑계대며 아이를 지운 무책임한 남자이며 논란3일만에 입장

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/AI/AI_study/data set_save/sbs.csv')

for row_index, row in data.iterrows():
    print(row_index)
    print(row['comment'])

for row_index, row in data.iterrows():
    #print(row_index)
    #print(row['comment'])

    sentence = row['comment']
    predict(sentence)
    print('\n')
    
    if predict(sentence) == 'bad':
        bad += 1
    else:
        good += 1
    
bad2 = bad
good2 = good

0
연예인은 이미지가 곧 생명 자체인데 이 정도면 끝났네. 이렇게 간단한 것을 아직까지 확인만 하고 있다니?
1
사실이라면 이미지 타격 크겠네... 두 사람이 같이 저지른 일인데 한사람만 비난받는건 좀... 여자 입장에선 한창 잘나가는데 낳고 결혼하고 싶었을수도 있겠지만 남자입장에선 낳자고 하기 어려웠을듯. 헤어졌다고 폭로해서 망하게 하는것도 바람직하진 않음
2
출연진 뒷담화했다. 헤어진 다음날 팬에게 싸인했더라. 촬영중엔 예민하니 주의해달라더라. 등등등 별의별걸로 다 까며 다시는 재기도 못하게 이불속 얘기까지 다 하는 악의적인 글이.. 몰카찍어 이별후 유포하는 넘들이랑 뭐가 다른가싶음
3
딘딘이가 사고 치지말라고 오래가자고 회마다 얘기하잖아 ~~~~좀 들으라구! 그 이쁜 얼굴에 왠 혼인빙자에 중절아가기 생긴것 부터 여자를 존중 안 했네 결혼해도 죄다
4
안타깝다 걍 결혼해도 인기가 좋았을 얼굴인데 .
5
대통령보다 연예인에게 도덕성을 더 요구하는 이상한 나라
6
혼인빙자, 낙태강요 너무 자극적인 단어 선택인 것 같은데요… 사귀면서 관계 안하는 연인 드물고피임을 안한 건지, 했는데도 아이가 생긴건진 모르겠지만..여튼, 제가 여자여도 이렇게까지 김선호씨가사회적으로 매장당해야할 일인지, ‘폭로’라는 걸 할만한 일인지 모르겠네요,,아이가 생겨도 낙태할 수 있고, 결혼을 안할수도 있는거니까요.. 결혼과 출산은 두 사람 모두의 바람과 축복 속에서 하는 거잖아요… 여튼 여자친구분 상처가 크실텐데 잘 치유하시고 김선호씨도 힘내시고, 원만하게 해결되길 빕니다
7
마약한것도아니고 성폭행한것도아니고 사기를 친것도아닌데...뭐 큰 범죄저지른거마냥 사실확인도 안된 일방적인 글하나에 다 물고뜯고난리...
8
지난 잘못된 둘만의 극히 개인적인 일을 상대가 잘 되니까 그게 배아파서 아님 다른 불순한 목적으로 폭로하는 사람도 솔직히 별로다~
9
연애하다 헤어진게 뭐그리 이슈라고,,,하차는 무슨!!!!괜찮다 힘내라
10
근데 김선호가 억지로강제로 끌고가서 낙태시킨것도 아니고, 결국은 본인결

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fbd661b0440>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1295, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = {'name':['middle','jtbc','sbs'],'bad':[bad,bad1,bad2], 'good':[good,good1,good2]}

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df

,name,bad,good
0,middle,2823,1239
1,jtbc,2228,1010
2,sbs,2823,1239
